In [ ]:
import os, sys
import gc

import matplotlib.pylab as plt
%pylab inline

import pandas as pd

from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
IMAGE_DIM = 3

# Read files, resize them (256,256,3) and store them as np.arrays

In [180]:
from tqdm import tnrange, tqdm_notebook

root = ".\Input\stage1_train"
dirlist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]

X_train = np.zeros((len(dirlist),IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DIM), dtype=np.uint8) 
y_train = np.zeros((len(dirlist),IMAGE_HEIGHT,IMAGE_WIDTH,1), dtype=np.bool)

print('Getting and resizing train images and masks ... ')
for i in tnrange(len(dirlist)):
    image_root = "{}\{}\images\{}.png".format(root, dirlist[i],dirlist[i])
    img = imread(image_root)[:,:,:IMAGE_DIM]
    img = resize(img, (IMAGE_HEIGHT, IMAGE_WIDTH), mode='constant', preserve_range=True)
    X_train[i] = img
    
    mask_root = "{}\{}\masks".format(root, dirlist[i])
    masklist = [item for item in os.listdir(mask_root) if os.path.isfile(os.path.join(mask_root, item))]

    mask = np.zeros((IMAGE_HEIGHT,IMAGE_WIDTH,1), dtype=np.bool)
    for msk in masklist:
        temp_mask = imread(mask_root+'\\'+msk)
        temp_mask = np.expand_dims(resize(temp_mask, (IMAGE_HEIGHT, IMAGE_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, temp_mask)
    y_train[i] = mask
   
root = ".\Input\stage1_test"
dirlist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]

X_test = np.zeros((len(dirlist), IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DIM), dtype=np.uint8)

print('Getting and resizing test images ... ')
for i in tnrange(len(dirlist)):
    image_root = "{}\{}\images\{}.png".format(root, dirlist[i], dirlist[i])
    img = imread(image_root)[:,:,:IMAGE_DIM]
    img = resize(img, (IMAGE_HEIGHT, IMAGE_WIDTH), mode='constant', preserve_range=True)
    X_test[i] = img

Getting and resizing train images and masks ... 



Getting and resizing test images ... 


In [182]:
#save the arrays
np.save("X_train", X_train)
np.save("y_train", y_train)
np.save("X_test", X_test)

In [183]:
t = np.load("X_train.npy")

array([[[[  4,   4,   4],
         [  4,   4,   4],
         [  4,   4,   4],
         ..., 
         [  3,   3,   3],
         [  2,   2,   2],
         [  2,   2,   2]],

        [[  5,   5,   5],
         [  5,   5,   5],
         [  5,   5,   5],
         ..., 
         [  4,   4,   4],
         [  3,   3,   3],
         [  2,   2,   2]],

        [[  5,   5,   5],
         [  5,   5,   5],
         [  5,   5,   5],
         ..., 
         [  4,   4,   4],
         [  3,   3,   3],
         [  2,   2,   2]],

        ..., 
        [[  2,   2,   2],
         [  3,   3,   3],
         [  3,   3,   3],
         ..., 
         [  2,   2,   2],
         [  2,   2,   2],
         [  2,   2,   2]],

        [[  2,   2,   2],
         [  2,   2,   2],
         [  3,   3,   3],
         ..., 
         [  2,   2,   2],
         [  2,   2,   2],
         [  1,   1,   1]],

        [[  2,   2,   2],
         [  1,   1,   1],
         [  2,   2,   2],
         ..., 
         [  2,   2,   2],
  

In [ ]:
# Get and resize train images and masks



sys.stdout.flush()

        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        print(mask_.shape)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

In [ ]:
from tqdm import tnrange, tqdm_notebook
from time import sleep

for i in tnrange(10, desc='1st loop'):
    for j in tqdm_notebook(range(100), desc='2nd loop'):
        pass

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(im[:,:,])
plt.subplot(122)
plt.imshow(ms)

In [ ]:
it = np.nditer(ms, flags=['multi_index'])
while not it.finished:
    if ms[it.multi_index] != 0:
        print ("%d <%s>" % (it[0], it.multi_index)),
    it.iternext() 